In [34]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [35]:
ex1 = pd.read_parquet('test/1.회원정보/2018_회원정보_전체_test.parquet')
ex2 = pd.read_parquet('test/2.신용정보/2018_신용정보_전체_test.parquet')
ex3 = pd.read_parquet('test/4.청구입금정보/2018_청구정보_전체.parquet')

In [36]:
all_cols = ex1.columns.tolist()
# print("전체 컬럼 이름:")
# print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
# print("\n문자형 컬럼 이름:")
# print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
# print("\n숫자형 컬럼 이름:")
# print(num_cols)

In [37]:
cols = ['가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분', '연회비발생카드수_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', 'Life_Stage']  # 실제 열 이름으로 바꿔주세요

In [38]:
mapping = {
    'L사': 0,
    'K사': 1,
    'S사': 2
}
ex1['가입통신회사코드'] = (
    ex1['가입통신회사코드']
      .map(mapping)        # L사→0, K사→1, S사→2, 그 외→NaN
      .fillna(-1)          # NaN→-1
      .astype(int)         # 정수형
)

In [39]:
ex1['연령'] = ex1['연령'].astype(str).str.replace('대', '', regex=False)

# 2) 숫자로 변환, 에러는 NaN→fillna(-1)→정수형
ex1['연령'] = (
    pd.to_numeric(ex1['연령'], errors='coerce')
      .fillna(-1)
      .astype(int)
)


In [40]:

regions = [
    '서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

region_type = CategoricalDtype(categories=regions, ordered=True)

for col in ['거주시도명', '직장시도명']:
    ex1[col] = (
        ex1[col]
           .astype(str)                   # 혹시 숫자 타입 섞여 있어도 문자열로
           .astype(region_type)           # 카테고리로 변환
           .cat.codes                     # 0~16, 리스트 외 값은 -1
    )

In [41]:
count_cols = [
    '연회비발생카드수_B0M',
    '상품관련면제카드수_B0M',
    '임직원면제카드수_B0M',
    '우수회원면제카드수_B0M',
    '기타면제카드수_B0M'
]

for col in count_cols:
    ex1[col] = (
        ex1[col]
          .astype(str)  # None → 'None', NaN → 'nan'
          .str.replace('개', '', regex=False)  # '0개' → '0', 'None' → 'None'
    )
    # 숫자로 변환, 에러는 NaN 처리 → NaN → -1 로 채우기 → 정수형
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [42]:
le_cols = ['_1순위신용체크구분', '_2순위신용체크구분', 'Life_Stage']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes 

In [43]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '남녀구분코드', '연령', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '거주시도명', '직장시도명', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용카드수_체크_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '이용금액_R3M_체크_가족', '_1순위카드이용금액', '_1순위카드이용건수', '_1순위신용체크구분', '_2순위카드이용금액', '_2순위카드이용건수', '_2순위신용체크구분', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비발생카드수_B0M', '연회비할인카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구

In [44]:
all_cols = ex2.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex2.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex2.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '최초한도금액', '카드이용한도금액', 'CA한도금액', '일시상환론한도금액', '월상환론한도금액', 'CA이자율_할인전', 'CL이자율_할인전', 'RV일시불이자율_할인전', 'RV현금서비스이자율_할인전', 'RV신청일자', 'RV약정청구율', 'RV최소결제비율', '자발한도감액횟수_R12M', '자발한도감액금액_R12M', '자발한도감액후경과월', '강제한도감액횟수_R12M', '강제한도감액금액_R12M', '강제한도감액후경과월', '한도증액횟수_R12M', '한도증액금액_R12M', '한도증액후경과월', '상향가능한도금액', '상향가능CA한도금액', '카드론동의여부', '월상환론상향가능한도금액', 'RV전환가능여부', '일시불ONLY전환가능여부', '카드이용한도금액_B1M', '카드이용한도금액_B2M', '특별한도보유여부_R3M', '연체감액여부_R3M', '한도심사요청건수', '한도요청거절건수', '한도심사요청후경과월', '한도심사거절후경과월', '시장단기연체여부_R6M', '시장단기연체여부_R3M', '시장연체상환여부_R6M', '시장연체상환여부_R3M', 'rv최초시작후경과일']

문자형 컬럼 이름:
['ID', '자발한도감액횟수_R12M', '한도증액횟수_R12M', '카드론동의여부', 'RV전환가능여부', '한도심사요청건수']

숫자형 컬럼 이름:
['기준년월', '최초한도금액', '카드이용한도금액', 'CA한도금액', '일시상환론한도금액', '월상환론한도금액', 'CA이자율_할인전', 'CL이자율_할인전', 'RV일시불이자율_할인전', 'RV현금서비스이자율_할인전', 'RV신청일자', 'RV약정청구율', 'RV최소결제비율', '자발한도감액금액_R12M', '자발한도감액후경과월', '강제한도감액횟수_R12M', '강제한도감액금액_R12M', '강제한도감액후경과월', '한도증액금액_R12M', '한도증액후경과월', '상향가능한도금액', '상향가능CA한도금액', '월상환론상향가능한도금액', '일시불ONLY

In [45]:
cols = ['자발한도감액횟수_R12M', '한도증액횟수_R12M', '카드론동의여부', 'RV전환가능여부', '한도심사요청건수']
print(ex2[cols].head())

  자발한도감액횟수_R12M 한도증액횟수_R12M 카드론동의여부 RV전환가능여부 한도심사요청건수
0            0회          0회       N        Z       0회
1            0회          0회       Y        N       0회
2            0회        1회이상       Y        Z       0회
3            0회          0회       Y        Z       0회
4            0회          0회       Y        Z       0회


In [46]:
count_cols2 = [
    '자발한도감액횟수_R12M',
    '한도증액횟수_R12M',
    '한도심사요청건수'
]

for col in count_cols2:
    # 1) 문자열로 바꾼 뒤 “회” 제거
    ex2[col] = ex2[col].astype(str).str.replace('회', '', regex=False)
    
    # 2) 숫자로 변환, 변환 불가(예: 'nan', 'None')는 NaN → fillna(-1) → 정수형
    ex2[col] = (
        pd.to_numeric(ex2[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [47]:
le_cols = ['카드론동의여부', 'RV전환가능여부']

for col in le_cols:
    codes, uniques = pd.factorize(ex2[col], sort=True)
    ex2[col] = codes


In [48]:
all_cols = ex2.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex2.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex2.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '최초한도금액', '카드이용한도금액', 'CA한도금액', '일시상환론한도금액', '월상환론한도금액', 'CA이자율_할인전', 'CL이자율_할인전', 'RV일시불이자율_할인전', 'RV현금서비스이자율_할인전', 'RV신청일자', 'RV약정청구율', 'RV최소결제비율', '자발한도감액횟수_R12M', '자발한도감액금액_R12M', '자발한도감액후경과월', '강제한도감액횟수_R12M', '강제한도감액금액_R12M', '강제한도감액후경과월', '한도증액횟수_R12M', '한도증액금액_R12M', '한도증액후경과월', '상향가능한도금액', '상향가능CA한도금액', '카드론동의여부', '월상환론상향가능한도금액', 'RV전환가능여부', '일시불ONLY전환가능여부', '카드이용한도금액_B1M', '카드이용한도금액_B2M', '특별한도보유여부_R3M', '연체감액여부_R3M', '한도심사요청건수', '한도요청거절건수', '한도심사요청후경과월', '한도심사거절후경과월', '시장단기연체여부_R6M', '시장단기연체여부_R3M', '시장연체상환여부_R6M', '시장연체상환여부_R3M', 'rv최초시작후경과일']

문자형 컬럼 이름:
['ID']

숫자형 컬럼 이름:
['기준년월', '최초한도금액', '카드이용한도금액', 'CA한도금액', '일시상환론한도금액', '월상환론한도금액', 'CA이자율_할인전', 'CL이자율_할인전', 'RV일시불이자율_할인전', 'RV현금서비스이자율_할인전', 'RV신청일자', 'RV약정청구율', 'RV최소결제비율', '자발한도감액횟수_R12M', '자발한도감액금액_R12M', '자발한도감액후경과월', '강제한도감액횟수_R12M', '강제한도감액금액_R12M', '강제한도감액후경과월', '한도증액횟수_R12M', '한도증액금액_R12M', '한도증액후경과월', '상향가능한도금액', '상향가능CA한도금액', '카드론동의여부', '월상환론상향가능한도금액', 'RV전환가능여부', '일시불ONLY전환가능여부', '카드

In [49]:
all_cols = ex3.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex3.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex3.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', '포인트_포인트_월적립_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액']

문자형 컬럼 이름:
['ID', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M']

숫자형 컬럼 이름:
['기준년월', '대표결제일', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '

In [50]:
cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M']
print(ex3[cols].head())

  대표결제방법코드 대표청구지고객주소구분코드 대표청구서수령지구분코드 청구서수령방법 할인건수_R3M 할인건수_B0M
0     자동이체           미확인          이메일     이메일    1회 이상    1회 이상
1     자동이체           미확인    당사페이앱+이메일   문자메세지    1회 이상    1회 이상
2     자동이체           미확인          이메일     이메일    1회 이상    1회 이상
3     자동이체           주거지           우편      우편    1회 이상    1회 이상
4     자동이체           주거지           우편      우편    1회 이상    1회 이상


In [51]:
print("할인건수_R3M unique values:")
print(ex3['할인건수_R3M'].unique())

print("\n할인건수_B0M unique values:")
print(ex3['할인건수_B0M'].unique())

할인건수_R3M unique values:
['1회 이상' '10회 이상' '20회 이상' '30회 이상' '40회 이상']

할인건수_B0M unique values:
['1회 이상' '10회 이상']


In [52]:
count_cols2 = ['할인건수_R3M', '할인건수_B0M']

for col in count_cols2:
    # 1) 문자열로 바꾼 뒤 “회” 제거
    ex3[col] = ex3[col].astype(str).str.replace('회 이상', '', regex=False)
    
    # 2) 숫자로 변환, 변환 불가(예: 'nan', 'None')는 NaN → fillna(-1) → 정수형
    ex3[col] = (
        pd.to_numeric(ex3[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [53]:
le_cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법']

for col in le_cols:
    codes, uniques = pd.factorize(ex3[col], sort=True)
    ex3[col] = codes


In [54]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex3.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex3.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '남녀구분코드', '연령', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '거주시도명', '직장시도명', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용카드수_체크_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '이용금액_R3M_체크_가족', '_1순위카드이용금액', '_1순위카드이용건수', '_1순위신용체크구분', '_2순위카드이용금액', '_2순위카드이용건수', '_2순위신용체크구분', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비발생카드수_B0M', '연회비할인카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구

In [55]:
df_merged = (
    ex1
    .merge(ex2,  on=['기준년월','ID'], how='left')
    .merge(ex3, on=['기준년월','ID'], how='left'))

In [56]:
cols = [
    '기준년월', 'ID', '남녀구분코드', '연령', '회원여부_이용가능_CA', '소지카드수_이용가능_신용', 
    '입회일자_신용', '회원여부_연체', '동의여부_한도증액안내', '수신거부여부_메일', '수신거부여부_SMS', 
    '가입통신회사코드', '최종탈회후경과월', '탈회횟수_발급6개월이내', '거주시도명', '마케팅동의여부', 
    '이용가능카드수_신용체크', '이용카드수_신용_가족', '이용금액_R3M_체크', '_1순위카드이용건수', 
    '_1순위신용체크구분', '_2순위카드이용건수', '최종카드발급일자', '이용여부_3M_해외겸용_본인', 
    '기본연회비_B0M', '제휴연회비_B0M', '카드신청건수', 'Life_Stage', '최초한도금액', 
    '일시상환론한도금액', '월상환론한도금액', 'CL이자율_할인전', 'RV약정청구율', 'RV최소결제비율', 
    '자발한도감액금액_R12M', '자발한도감액후경과월', '강제한도감액금액_R12M', '강제한도감액후경과월', 
    '한도증액후경과월', '상향가능CA한도금액', '월상환론상향가능한도금액', '일시불ONLY전환가능여부', 
    '카드이용한도금액_B2M', '연체감액여부_R3M', '대표결제일', '대표청구지고객주소구분코드', 
    '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R6M', '청구금액_B0', 
    '포인트_포인트_건별_B0M', '포인트_포인트_월적립_B0M', '포인트_잔여포인트_B0M', 
    '마일_적립포인트_R12M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인건수_B0M', 
    '할인금액_B0M', '선결제건수_R3M', '연체건수_R3M'
]

df_selected = df_merged[cols]

In [57]:
df_selected.to_parquet('회원_신용_청구_test.parquet', index=False)